----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content - Pricing - Streaming

This notebook demonstrates using a Pricing object to 
- receive streaming real-time data
- store + update the values in a Pandas Dataframe.  

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
from refinitiv.data.content import pricing
import refinitiv.data as rd
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [3]:
rd.open_session()

<refinitiv.data.session.platform.Definition object at 0x173dd8b9520 {session_name='rdp'}>

### Define a callback to receive streaming data

In [4]:
# declare a DataFrame to use for storing our streaming data
df = DataFrame()

# Function to initially populate Dataframe, once initial values received for all items
def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)

# Function to update dataframe, when we receive updates for individual items
def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)
    

### Create streams for our RICs and open them

In [5]:
# Define our Streaming Price object
streams = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()

# Callback for when we have received initial values for all items
streams.on_complete(create_dataframe)
# Specify callback handler for any updates
streams.on_update(update_dataframe)

# Send the requests to the server and open the streams for all items
streams.open()

,BID,ASK
EUR=,1.1294,1.1297
GBP=,1.3337,1.334
JPY=,113.87,113.88
CAD=,1.2761,1.2765


## Close the stream

In [6]:
streams.close()

<OpenState.Closed: 3>

## Close the session

In [7]:
rd.close_session()